In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import scipy
from pathlib import Path
from scipy.signal import lfilter, firwin

sys.path.append('../src/')
from utils import analysis_utils as analysis
from utils import plotting_utils as plotting
from utils import processing
## Data settings
from statannot import add_stat_annotation
from matplotlib.patches import Rectangle

## Plotting settings
import matplotlib.patches as mpatches
import seaborn as sns
import matplotlib.pyplot as plt

plt.rcParams['font.sans-serif'] = "Arial"
plt.rcParams['font.family'] = "sans-serif"

sns.set_context("talk")

In [ ]:
# def add_position(df: pd.DataFrame, position: data_io.DataStream):

#     df = pd.merge_asof(df.sort_index(), position.data.sort_index(), direction='nearest', on="Seconds").set_index("Seconds").sort_index()
#     df.columns = [*df.columns[:-1], 'Position']
#     return df

In [ ]:
df_summary = pd.DataFrame()

# start_dir = r"Z:\scratch\vr-foraging\data\672106"  # Starting directory to search
# filename_part = '20231108T'  # Part of the filename you're searching for
# session_path = find_file(start_dir, filename_part)

session_path = r"Z:\scratch\vr-foraging\data\699899\20240319T084500"  # Starting directory to search
session = session_path[-15:-7]
mouse = session_path[-22:-16]
session_path = Path(session_path)

data = analysis.load_session_data(session_path)

if 'harp_olfactometer' in data:
    data['harp_olfactometer'].streams.OdorValveState.load_from_file()
    data['harp_olfactometer'].streams.EndValveState.load_from_file()

data['harp_behavior'].streams.OutputSet.load_from_file()
data['harp_behavior'].streams.OutputClear.load_from_file()
data['config'].streams.TaskLogic.load_from_file()

reward_sites, active_site, encoder, config = analysis.parse_data(data)

patch_limit = analysis.choose_cut(reward_sites, 10)
reward_sites = reward_sites.loc[reward_sites.active_patch <= patch_limit]

In [ ]:
# Load treadmill data
# Maybe look at how the traces change with these two way
encoder = encoder['Encoder']

treadmill_metadata = config.streams.Rig.data["treadmill"]
try:
    converter = treadmill_metadata["wheelDiameter"] * np.pi / treadmill_metadata["pulsesPerRevolution"] * (-1 if treadmill_metadata["invertDirection"] else 1)
except:
    converter = treadmill_metadata["wheel_diameter"] * np.pi / treadmill_metadata["pulses_per_revolution"] * (-1 if treadmill_metadata["invert_direction"] else 1)

encoder = encoder.apply(lambda x : x * converter)

encoder.index = pd.to_datetime(encoder.index, unit="s")
encoder = encoder.resample("33ms").sum().interpolate(method="linear") / 0.033
encoder.index = (encoder.index - pd.to_datetime(0))
encoder.index = encoder.index.total_seconds()

In [ ]:
# win = (0, 1500)
win = (1000,1100)

show_speed = True
save_name = "session_w_speed"


In [ ]:
data['harp_behavior'].streams.OutputSet.load_from_file()
data['harp_behavior'].streams.PulseSupplyPort0.load_from_file() # Duration of each pulse
data['harp_behavior'].streams.DigitalInputState.load_from_file()

# Find reward sites
sites = data['software_events'].streams.ActiveSite.data

zero_index = sites.index[0]

# Find ChoiceFeedback events (i.e. successful stops)
choice_feedback = data['software_events'].streams.ChoiceFeedback.data

# Check for licks
## mask for digital inputs

di_state = data['harp_behavior'].streams.DigitalInputState.data['DIPort0']
lick_onset = di_state.loc[di_state == True]

# Find give reward event
give_reward = data['harp_behavior'].streams.OutputSet.data[['SupplyPort0']]
give_reward = give_reward.loc[give_reward.SupplyPort0 == True]

# Find hardware reward events
pulse_duration = data['harp_behavior'].streams.PulseSupplyPort0.data
valve_output_pulse = data['harp_behavior'].streams.OutputSet.data['DOPort0']

label_dict = {
    "InterSite": '#808080',
    "Odor 1": '#d95f02',
    "Odor 2": '#1b9e77',
    "Odor 3": '#7570b3',
    "InterPatch": '#b3b3b3'}

fig, axs = plt.subplots(1,1, figsize=(20,4))

_legend = {}
done = True
for idx, site in enumerate(sites.iloc[:-1].iterrows()):
    site_label = site[1]['data']["label"]
    if site_label == "Reward":
        site_label = f"Odor {site[1]['data']['odor']['index']+1}"
        facecolor = label_dict[site_label]
    elif site_label == "RewardSite":
        site_label = f"Odor {site[1]['data']['odor_specification']['index']+1}"
        facecolor = label_dict[site_label]
    elif site_label == "InterPatch":
        facecolor = label_dict[site_label]
    else:
        site_label = "InterSite"
        facecolor = label_dict["InterSite"]

    p = Rectangle(
        (sites.index[idx] - zero_index, -2), sites.index[idx+1] - sites.index[idx], 8,
        linewidth = 0, facecolor = facecolor, alpha = .5)
    _legend[site_label] = p
    axs.add_patch(p)

s, lw = 400, 2
# Plotting raster
y_idx = -0.4
_legend["Choice Tone"] = axs.scatter(choice_feedback.index - zero_index+0.2,
           choice_feedback.index * 0 + y_idx,
           marker="s", s=100, lw=lw, c='darkblue',
           label="Choice Tone")
y_idx += 1
_legend["Lick"] = axs.scatter(lick_onset.index - zero_index,
           lick_onset.index * 0 + y_idx,
           marker="|", s=s, lw=lw, c='k',
           label="Lick")
_legend["Reward"] = axs.scatter(give_reward.index - zero_index,
           give_reward.index*0 + y_idx,
           marker=".", s=s, lw=lw, c='deepskyblue',
           label="Reward")
y_idx += 1

#ax.set_xticks(np.arange(0, sites.index[-1] - zero_index, 10))
axs.set_yticklabels([])
axs.set_xlabel("Time(s)")
axs.set_ylim(bottom=-1, top = 3)
axs.grid(False)
axs.set_xlim(win)

if show_speed:
    ax2 = axs.twinx()
    _legend["Velocity"] = ax2.plot(encoder.index - zero_index, encoder, c="k", label="Encoder", alpha = 0.8)[0]
    try:
        v_thr = config.streams.TaskLogic.data["operationControl"]["positionControl"]["stopResponseConfig"]["velocityThreshold"]
    except:
        v_thr = 8
    _legend["Stop Threshold"] = ax2.plot(ax2.get_xlim(), (v_thr, v_thr), c="k", label="Encoder", alpha = 0.5, lw = 2, ls = "--")[0]
    ax2.grid(False)
    ax2.set_ylim((-5, 70))
    ax2.set_ylabel("Velocity (cm/s)")
axs.legend(_legend.values(), _legend.keys(), bbox_to_anchor=(1.05, 0.5), loc='center left', borderaxespad=0.)

# axs[0].stairs(software_events.streams.RewardAvailableInPatch.data["data"].values[:-1],
#           software_events.streams.RewardAvailableInPatch.data["data"].index.values -  zero_index,
#           lw = 3, color = 'k', fill=0)
axs.set_xlabel("Time(s)")
axs.grid(False)
axs.set_xlim(win)
# axs.get_xaxis().set_visible(False)
axs.set_ylim(bottom=-1, top = 4)
axs.set_yticks([0,3])
axs.yaxis.tick_right()

# if save_name is not None:
#     plt.savefig(janelia_figures + f"\{save_name}_time.svg", bbox_inches='tight', pad_inches=0.1, transparent=True)
plt.show()

In [ ]:
# Start plotting raster
operation_control.streams.CurrentPosition.load_from_file()
position = operation_control.streams.CurrentPosition

fig, ax = plt.subplots(figsize=(25,5))
_legend = {}

_sites = add_position(sites, position=position)

for idx, site in enumerate(sites.iloc[:-1].iterrows()):
    site_label = site[1]["data"]["label"]
    if site_label == "Reward":
        site_label = f"{site_label}_{site[1]['data']['odor']['index']}"
        facecolor = label_dict[site_label]
    else:
        facecolor = label_dict[site_label]

    p = Rectangle(
        (_sites["Position"].iloc[idx], -2), _sites["Position"].iloc[idx+1] - _sites["Position"].iloc[idx], 8,
        linewidth = 0, facecolor = facecolor, alpha = 0.5 )
    _legend[site_label] = p
    ax.add_patch(p)

s, lw = 400, 2
# Plotting raster
y_idx = 0
_legend["ChoiceFeedback"] = ax.scatter(add_position(choice_feedback, position=position)["Position"].values,
           choice_feedback.index * 0 + y_idx,
           marker=".", s=s, lw=lw, c='#e6ab02',
           label="ChoiceFeedback")
y_idx += 1
_legend["Lick"] = ax.scatter(add_position(lick_onset, position=position)["Position"].values,
           lick_onset.index * 0 + y_idx,
           marker="|", s=s, lw=lw, c='k',
           label="Lick")
_legend["ValveOpen"] = ax.scatter(add_position(valve_output_pulse, position=position)["Position"].values,
           valve_output_pulse.index*0 + y_idx,
           marker=".", s=s, lw=lw, c='#ff7f00',
           label="ValveOpen")
y_idx += 1

ax.set_yticklabels([])
ax.set_xlabel("VrSpace(cm)")
ax.set_xlim(_sites.loc[(_sites.index - zero_index > win[0]) & (_sites.index - zero_index < win[1]), :]["Position"].values[[0, -1]])
ax.set_ylim(bottom=-1, top = 3)
ax.grid(False)

if show_speed:
    ax2 = ax.twinx()
    _legend["Velocity"] = ax2.plot(add_position(encoder, position=position)["Position"].values, encoder.values, c="k", label="Encoder", alpha = 0.8)[0]
    v_thr = config.streams.TaskLogic.data["operationControl"]["positionControl"]["stopResponseConfig"]["velocityThreshold"]
    _legend["Velocity_Threshold"] = ax2.plot(ax2.get_xlim(), (v_thr, v_thr), c="k", label="Encoder", alpha = 0.5, lw = 2, ls = "--")[0]
    ax2.grid(False)
    ax2.set_ylim((-5, 50))
    ax2.set_ylabel("Velocity (cm/s)")
ax.legend(_legend.values(), _legend.keys(), bbox_to_anchor=(1.05, 0.5), loc='center left', borderaxespad=0.)
if save_name is not None:
    plt.savefig(f"{save_name}_space.svg", bbox_inches='tight', pad_inches=0.1, transparent=True)
plt.show()




In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(15, 5))
sites["label"] = sites.apply(lambda x : x["data"]["label"], axis=1)
for i, (label, group) in enumerate(sites.groupby("label")):
    size = group.apply(lambda x : x["data"]["length"], axis=1)
    axs[i].hist(size.values, bins= np.arange(30, 100, 2.5), density = 1)
    axs[i].vlines(size.mean(), 0, 0.1, color="r", label="Mean")
    axs[i].set_title(f"{label} // Mean: {size.mean():.2f}")
    axs[i].set_xlabel("Length (cm)")
    if i == 0:
        axs[i].set_ylabel("Density")
    axs[i].set_xlim((30, 100))
plt.show()

In [ ]:
# align on site entry


window = (-0.5, 1)
for label, label_group in sites.groupby("label"):
    fig, axs = plt.subplots(2,1)
    dist = []
    for index, site in label_group.iterrows():
        this_window = np.array((index - window[0], index + window[1]))
        trace = encoder.loc[index + window[0] : index + window[1]]
        dist.append(encoder.loc[index + window[0] : index].values.mean())
        axs[1].plot(trace.index - index, trace.values, color = 'k', alpha=0.01)

    axs[1].set_xlabel("Time (s)")
    axs[1].set_ylabel("Speed(cm/s)")
    axs[1].set_ylim((-10, 60))

    axs[1].hist(np.array(dist), bins=np.arange(-10, 60, 2.5), density=1)
    axs[1].vlines(np.array(dist).mean(), 0, 0.1, color="r", label="Mean")
    axs[1].set_xlabel(f"Average speed (cm/s) @ {window[0]}:{0}")
    fig.suptitle(label)
